In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import itertools
import ast
from ast import literal_eval
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /nfshome/tc1767/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /nfshome/tc1767/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /nfshome/tc1767/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Get data for 2004

In [2]:
pat0401 = pd.read_csv('/projects/cps2019_funding/shared/Patents_Data/patentData/patents_200401')
pat0405 = pd.read_csv('/projects/cps2019_funding/shared/Patents_Data/patentData/patents_200405')
pat0409 = pd.read_csv('/projects/cps2019_funding/shared/Patents_Data/patentData/patents_200409')
pat04 = pd.concat([pat0401, pat0405, pat0409]).reset_index(drop=True)

In [3]:
pat04.head()

,Unnamed: 0,IPCs,applications,assignees,cited_patents,cpcs,nbers,patent_abstract,patent_id,patent_type,uspcs,wipos
0,0,"[{u'ipc_section': u'H', u'ipc_subclass': u'R',...","[{u'app_type': u'10', u'app_id': u'10/592576'}]","[{u'assignee_key_id': u'487946', u'assignee_ci...",[{u'cited_patent_category': u'cited by examine...,[{u'cpc_subgroup_title': u'Stereophonic arrang...,"[{u'nber_category_title': None, u'nber_subcate...","A Closed Loop Headphone Apparatus with 1, 2, 3...",10028058,utility,"[{u'uspc_mainclass_title': None, u'uspc_sequen...",[{u'wipo_field_title': u'Audio-visual technolo...
1,1,"[{u'ipc_section': u'C', u'ipc_subclass': u'N',...","[{u'app_type': u'10', u'app_id': u'10/829504'}]","[{u'assignee_key_id': u'129714', u'assignee_ci...",[{u'cited_patent_category': u'cited by applica...,[{u'cpc_subgroup_title': u'Mutation or genetic...,"[{u'nber_category_title': None, u'nber_subcate...",Materials and methods are provided for produci...,10100316,utility,"[{u'uspc_mainclass_title': None, u'uspc_sequen...","[{u'wipo_field_title': u'Biotechnology', u'wip..."
2,2,"[{u'ipc_section': u'A', u'ipc_subclass': u'K',...","[{u'app_type': u'10', u'app_id': u'10/795128'}]","[{u'assignee_key_id': u'158959', u'assignee_ci...",[{u'cited_patent_category': u'cited by applica...,[{u'cpc_subgroup_title': u'Medicinal preparati...,"[{u'nber_category_title': None, u'nber_subcate...",This invention provides methods and kits for m...,10105417,utility,"[{u'uspc_mainclass_title': None, u'uspc_sequen...","[{u'wipo_field_title': u'Pharmaceuticals', u'w..."
3,3,"[{u'ipc_section': u'B', u'ipc_subclass': u'D',...","[{u'app_type': u'10', u'app_id': u'10/821311'}]","[{u'assignee_key_id': u'152087', u'assignee_ci...",[{u'cited_patent_category': u'cited by applica...,[{u'cpc_subgroup_title': u'Processes for apply...,"[{u'nber_category_title': None, u'nber_subcate...",Devices and methods are provided for ejecting ...,10112212,utility,"[{u'uspc_mainclass_title': None, u'uspc_sequen...","[{u'wipo_field_title': u'Surface technology, c..."
4,4,"[{u'ipc_section': u'G', u'ipc_subclass': u'Q',...","[{u'app_type': u'10', u'app_id': u'10/833495'}]","[{u'assignee_key_id': u'24540', u'assignee_cit...",[{u'cited_patent_category': u'cited by examine...,[{u'cpc_subgroup_title': u'Finance; Insurance;...,"[{u'nber_category_title': None, u'nber_subcate...",A method and apparatus to manage accounts audi...,10115152,utility,"[{u'uspc_mainclass_title': None, u'uspc_sequen...",[{u'wipo_field_title': u'IT methods for manage...


In [4]:
pat04.shape

(99144, 12)

### City name with number of patents in descending order

In [5]:
#output city matched with its index number so that we can use the index number to extract rows from dataframe 
#for different cities
def find_cities(df):
    for i in range(len(df)):
        try:
            yield (i, literal_eval(pat04['assignees'][i])[0]['assignee_city'])
        except SyntaxError:
            yield (i, 'TBD')

In [6]:
#output a list of tuples with unique cities and corresponding number of patents
def count_cities(l):
    counts = {}
    for tup in l:
        counts[tup[1]] = counts.get(tup[1], 0) + 1
    return sorted(counts.items(), key=lambda x: x[1], reverse=True)

In [7]:
cities = list(find_cities(pat04))
cities_counts = count_cities(cities)

In [8]:
str(cities[0][1])

'Vallejo'

In [9]:
cities_counts[:10]

[('Santa Clara', 4293),
 ('Armonk', 3956),
 ('Houston', 3700),
 ('Redmond', 3149),
 ('San Jose', 2861),
 ('Boise', 1686),
 ('Wilmington', 1612),
 ('Sunnyvale', 1586),
 ('Irvine', 1362),
 ('New York', 1318)]

In [10]:
#top 10 cities
[x[0] for x in cities_counts][:10]

['Santa Clara',
 'Armonk',
 'Houston',
 'Redmond',
 'San Jose',
 'Boise',
 'Wilmington',
 'Sunnyvale',
 'Irvine',
 'New York']

### Dataframe and Words From Abstract for specific city

In [11]:
def df_city(df, city):
    return df.iloc[list(map(lambda x: x[0], filter(lambda x: x[1]==city, cities)))]

In [39]:
def tokenize_words(df):
    stop_words = stopwords.words('english')
    allwords = []
    for i in list(df.index):
        try:
            words = [x.lower() for x in nltk.word_tokenize(df.loc[i]['patent_abstract']) if x.isalnum()]
            allwords.append([word for word in words if not word in stop_words])
        except:
            continue
    return list(itertools.chain.from_iterable(allwords))

In [45]:
def words_dict(df):
    rawcounts = dict(Counter([x for x in tokenize_words(df)]))
    pos = dict(nltk.pos_tag(rawcounts.keys()))    
    d = {}
    for key in rawcounts.keys():
        try:
            d.setdefault(key,[]).append(rawcounts[key])        
        except KeyError:
            pass

        try:
            d.setdefault(key,[]).append(pos[key])          
        except KeyError:
            pass
    return d

In [48]:
#input dictionary and pos that we want to extract
#see https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/ for complete POS-tag list
def extract_by_pos(d, pos):
    for k,v in d.items():
        if pos not in v[1]:
            continue
        yield k,v    

#### Santa Clara

In [41]:
cities_counts[0][0]

'Santa Clara'

In [42]:
pat0401 = df_city(pat04, cities_counts[0][0])

In [43]:
pat0401[pat0401['patent_abstract'].isnull()]

,Unnamed: 0,IPCs,applications,assignees,cited_patents,cpcs,nbers,patent_abstract,patent_id,patent_type,uspcs,wipos
28832,8832,"[{u'ipc_section': u'D', u'ipc_subclass': u'402...","[{u'app_type': u'29', u'app_id': u'29/201574'}]","[{u'assignee_key_id': u'13780', u'assignee_cit...",[{u'cited_patent_category': u'cited by other'}...,"[{u'cpc_subgroup_title': None, u'cpc_category'...","[{u'nber_category_title': None, u'nber_subcate...",NaN,D496365,design,"[{u'uspc_mainclass_title': u'Recording, commun...","[{u'wipo_field_title': None, u'wipo_sector_tit..."
29698,9698,"[{u'ipc_section': None, u'ipc_subclass': None,...","[{u'app_type': u'29', u'app_id': u'29/200582'}]","[{u'assignee_key_id': u'182526', u'assignee_ci...",[{u'cited_patent_category': u'cited by examine...,"[{u'cpc_subgroup_title': None, u'cpc_category'...","[{u'nber_category_title': None, u'nber_subcate...",NaN,D504895,design,"[{u'uspc_mainclass_title': u'Recording, commun...","[{u'wipo_field_title': None, u'wipo_sector_tit..."
62925,1224,"[{u'ipc_section': None, u'ipc_subclass': None,...","[{u'app_type': u'29', u'app_id': u'29/208829'}]","[{u'assignee_key_id': u'337454', u'assignee_ci...",[{u'cited_patent_category': u'cited by examine...,"[{u'cpc_subgroup_title': None, u'cpc_category'...","[{u'nber_category_title': None, u'nber_subcate...",NaN,D515086,design,"[{u'uspc_mainclass_title': u'Recording, commun...","[{u'wipo_field_title': None, u'wipo_sector_tit..."
96853,2505,"[{u'ipc_section': None, u'ipc_subclass': None,...","[{u'app_type': u'29', u'app_id': u'29/213042'}]","[{u'assignee_key_id': u'13780', u'assignee_cit...",[{u'cited_patent_category': u'cited by examine...,"[{u'cpc_subgroup_title': None, u'cpc_category'...","[{u'nber_category_title': None, u'nber_subcate...",NaN,D515574,design,"[{u'uspc_mainclass_title': u'Recording, commun...","[{u'wipo_field_title': None, u'wipo_sector_tit..."


In [46]:
#dictionary for words in year 04 and city 01 (with top number of patents)
#each with its frequency and POS tag
words0401 = words_dict(pat0401[~pat0401['patent_abstract'].isnull()])
words0401

{'memory': [1563, 'NN'],
 'array': [345, 'NN'],
 'includes': [2037, 'VBZ'],
 'plurality': [1131, 'NN'],
 'several': [48, 'JJ'],
 'levels': [76, 'NNS'],
 'forming': [303, 'VBG'],
 'alternating': [24, 'VBG'],
 'cells': [140, 'NNS'],
 'formed': [568, 'VBN'],
 'intersection': [5, 'NN'],
 'plane': [100, 'NN'],
 'oriented': [28, 'VBN'],
 'direction': [96, 'NN'],
 'relative': [141, 'JJ'],
 'substrate': [1387, 'NN'],
 'serial': [60, 'NN'],
 'chain': [39, 'NN'],
 'diode': [49, 'JJ'],
 'stack': [144, 'NN'],
 'certain': [59, 'JJ'],
 'embodiments': [784, 'NNS'],
 'row': [65, 'VB'],
 'column': [69, 'NN'],
 'circuits': [191, 'NNS'],
 'arranged': [175, 'VBD'],
 'interchange': [1, 'NN'],
 'function': [216, 'NN'],
 'depending': [33, 'VBG'],
 'upon': [248, 'IN'],
 'directionality': [3, 'NN'],
 'selected': [324, 'VBN'],
 'drivers': [22, 'NNS'],
 'capable': [131, 'JJ'],
 'passing': [28, 'VBG'],
 'write': [185, 'JJ'],
 'current': [724, 'JJ'],
 'either': [122, 'CC'],
 'cell': [191, 'NN'],
 'preferred': [44,

In [49]:
words0401_NN = sorted(list(extract_by_pos(words0401, 'NN')),key=lambda x: x[1][0], reverse=True)
words0401_NN

[('data', [2661, 'NNS']),
 ('system', [2495, 'NN']),
 ('method', [2305, 'NN']),
 ('device', [1885, 'NN']),
 ('circuit', [1672, 'NN']),
 ('memory', [1563, 'NN']),
 ('layer', [1390, 'NN']),
 ('substrate', [1387, 'NN']),
 ('embodiment', [1296, 'NN']),
 ('plurality', [1131, 'NN']),
 ('invention', [899, 'NN']),
 ('surface', [866, 'NN']),
 ('voltage', [820, 'NN']),
 ('embodiments', [784, 'NNS']),
 ('network', [784, 'NN']),
 ('processor', [779, 'NN']),
 ('power', [763, 'NN']),
 ('information', [752, 'NN']),
 ('control', [748, 'NN']),
 ('process', [698, 'NN']),
 ('output', [696, 'NN']),
 ('portion', [681, 'NN']),
 ('source', [659, 'NN']),
 ('semiconductor', [629, 'NN']),
 ('input', [616, 'NN']),
 ('unit', [590, 'NN']),
 ('address', [589, 'NN']),
 ('methods', [583, 'NNS']),
 ('packet', [561, 'NN']),
 ('computer', [553, 'NN']),
 ('value', [543, 'NN']),
 ('cache', [530, 'NN']),
 ('devices', [514, 'NNS']),
 ('structure', [495, 'NN']),
 ('region', [468, 'NN']),
 ('time', [452, 'NN']),
 ('clock', [4

In [50]:
words0401_JJ = sorted(list(extract_by_pos(words0401, 'JJ')),key=lambda x: x[1][0], reverse=True)
words0401_JJ

[('second', [2195, 'JJ']),
 ('signal', [1706, 'JJ']),
 ('least', [1209, 'JJS']),
 ('current', [724, 'JJ']),
 ('material', [722, 'JJ']),
 ('present', [587, 'JJ']),
 ('optical', [524, 'JJ']),
 ('multiple', [426, 'JJ']),
 ('wherein', [419, 'JJ']),
 ('node', [382, 'JJ']),
 ('metal', [373, 'JJ']),
 ('virtual', [367, 'JJ']),
 ('instruction', [364, 'JJ']),
 ('different', [340, 'JJ']),
 ('high', [335, 'JJ']),
 ('logic', [327, 'JJ']),
 ('conductive', [327, 'JJ']),
 ('light', [324, 'JJ']),
 ('user', [306, 'JJ']),
 ('element', [303, 'JJ']),
 ('silicon', [289, 'JJ']),
 ('low', [282, 'JJ']),
 ('stored', [282, 'JJ']),
 ('determine', [278, 'JJ']),
 ('digital', [276, 'JJ']),
 ('electrical', [275, 'JJ']),
 ('thermal', [263, 'JJ']),
 ('disposed', [261, 'JJ']),
 ('single', [258, 'JJ']),
 ('generated', [249, 'JJ']),
 ('physical', [233, 'JJ']),
 ('stage', [223, 'JJ']),
 ('filter', [212, 'JJ']),
 ('amplifier', [211, 'JJ']),
 ('register', [200, 'JJ']),
 ('thread', [200, 'JJ']),
 ('interconnect', [194, 'JJ'])

#### Boise

In [27]:
cities_counts[5][0]

'Boise'

In [28]:
pat0406 = df_city(pat04, cities_counts[5][0])

In [29]:
words0406 = words_dict(pat0406[~pat0406['patent_abstract'].isnull()])
words0406

{'the': [16035, 'DT'],
 'invention': [544, 'NN'],
 'includes': [1173, 'VBZ'],
 'a': [11680, 'DT'],
 'method': [919, 'NN'],
 'of': [6902, 'IN'],
 'forming': [662, 'VBG'],
 'conductive': [754, 'JJ'],
 'interconnect': [141, 'NN'],
 'an': [2150, 'DT'],
 'electrical': [275, 'JJ'],
 'node': [126, 'NN'],
 'location': [113, 'NN'],
 'is': [3253, 'VBZ'],
 'defined': [51, 'VBN'],
 'to': [4865, 'TO'],
 'be': [1263, 'VB'],
 'supported': [22, 'VBN'],
 'by': [951, 'IN'],
 'substrate': [1394, 'JJ'],
 'silicide': [56, 'NN'],
 'formed': [994, 'VBD'],
 'in': [2899, 'IN'],
 'contact': [409, 'NN'],
 'with': [1397, 'IN'],
 'exposing': [41, 'VBG'],
 'hydrogen': [28, 'NN'],
 'ticl4': [2, 'NN'],
 'and': [5301, 'CC'],
 'plasma': [82, 'JJ'],
 'conditions': [84, 'NNS'],
 'cause': [22, 'VBP'],
 'ti': [3, 'NN'],
 'from': [996, 'IN'],
 'combine': [10, 'JJ'],
 'silicon': [377, 'NN'],
 'form': [270, 'NN'],
 'tisix': [4, 'NN'],
 'conductively': [12, 'RB'],
 'doped': [123, 'VBD'],
 'material': [1186, 'NN'],
 'over': [60

In [30]:
words0406_NN = sorted(list(extract_by_pos(words0406, 'NN')),key=lambda x: x[1][0], reverse=True)
words0406_NN

[('layer', [1952, 'NN']),
 ('memory', [1949, 'NN']),
 ('material', [1186, 'NN']),
 ('device', [1017, 'NN']),
 ('semiconductor', [984, 'NN']),
 ('method', [919, 'NN']),
 ('circuit', [801, 'NN']),
 ('surface', [761, 'NN']),
 ('gate', [733, 'NN']),
 ('data', [668, 'NNS']),
 ('region', [662, 'NN']),
 ('dielectric', [626, 'NN']),
 ('voltage', [569, 'NN']),
 ('invention', [544, 'NN']),
 ('plurality', [521, 'NN']),
 ('methods', [509, 'NNS']),
 ('line', [498, 'NN']),
 ('devices', [488, 'NNS']),
 ('transistor', [461, 'NN']),
 ('metal', [440, 'NN']),
 ('cell', [432, 'NN']),
 ('structure', [427, 'NN']),
 ('contact', [409, 'NN']),
 ('system', [392, 'NN']),
 ('lines', [386, 'NNS']),
 ('silicon', [377, 'NN']),
 ('portion', [374, 'NN']),
 ('cells', [329, 'NNS']),
 ('control', [327, 'NN']),
 ('output', [325, 'NN']),
 ('clock', [305, 'NN']),
 ('capacitor', [302, 'NN']),
 ('circuitry', [288, 'NN']),
 ('bit', [274, 'NN']),
 ('form', [270, 'NN']),
 ('signals', [268, 'NNS']),
 ('delay', [265, 'NNS']),
 ('l

In [31]:
words0406_JJ = sorted(list(extract_by_pos(words0406, 'JJ')),key=lambda x: x[1][0], reverse=True)
words0406_JJ

[('substrate', [1394, 'JJ']),
 ('second', [1319, 'JJ']),
 ('conductive', [754, 'JJ']),
 ('least', [688, 'JJS']),
 ('signal', [638, 'JJ']),
 ('such', [588, 'JJ']),
 ('die', [496, 'JJ']),
 ('embodiment', [395, 'JJ']),
 ('array', [390, 'JJ']),
 ('other', [316, 'JJ']),
 ('more', [285, 'JJR']),
 ('microelectronic', [277, 'JJ']),
 ('electrical', [275, 'JJ']),
 ('current', [226, 'JJ']),
 ('test', [222, 'JJS']),
 ('high', [214, 'JJ']),
 ('present', [212, 'JJ']),
 ('element', [211, 'JJ']),
 ('configured', [193, 'JJ']),
 ('active', [187, 'JJ']),
 ('side', [184, 'JJ']),
 ('adjacent', [172, 'JJ']),
 ('different', [164, 'JJ']),
 ('low', [152, 'JJ']),
 ('select', [150, 'JJ']),
 ('vertical', [149, 'JJ']),
 ('logic', [139, 'JJ']),
 ('multiple', [129, 'JJ']),
 ('same', [125, 'JJ']),
 ('magnetic', [125, 'JJ']),
 ('exposed', [123, 'JJ']),
 ('write', [123, 'JJ']),
 ('improved', [118, 'JJ']),
 ('selectively', [117, 'JJ']),
 ('workpiece', [117, 'JJ']),
 ('local', [115, 'JJ']),
 ('reduced', [112, 'JJ']),
 ('